In [112]:
import boto3
import pickle
import pandas as pd
from io import StringIO
import io

!pip install gensim
from gensim.models.word2vec import Word2Vec
from sklearn.cluster import KMeans;
from sklearn.neighbors import KDTree
import numpy as np

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [99]:
s3 = boto3.client('s3')

object = s3.get_object(Bucket='inroads-test-bucket1',Key='data/Vectorized/inroads_vectorized.csv')
inroads_vectorized = pd.read_csv(io.BytesIO(object['Body'].read()), encoding='utf8', index_col=0)

inroads_vectorized.shape

(10061, 302)

In [4]:
s3 = boto3.client('s3')

object = s3.get_object(Bucket='inroads-test-bucket1',Key='data/Vectorized/old_vectorized.csv')
old_vectorized = pd.read_csv(io.BytesIO(object['Body'].read()), encoding='utf8', index_col=0)

old_vectorized.shape

(941, 302)

In [103]:
inroads_vectorized.columns

Index(['tokens', 'vector', '0', '1', '2', '3', '4', '5', '6', '7',
       ...
       '290', '291', '292', '293', '294', '295', '296', '297', '298', '299'],
      dtype='object', length=302)

In [6]:
## extract details

In [60]:
def Convert(string): 
    li = list(string.split(" ")) 
    return li

In [ ]:
## old data

In [18]:
old_vocab = list()

for i in range(0,len(old_vectorized)):
    test = old_vectorized.iloc[i].token
    old_vocab.append(test)

old_vocab_df = pd.DataFrame(old_vocab, columns=["token"])

In [90]:
old_vectors_2 = list()

for i in range(0,len(old_vectorized)):
    test = old_vectorized.iloc[i].vector
    test_2 = str(test)[1:-1] 
    test_2 = Convert(test_2)
    while("" in test_2) : 
        test_2.remove("")
    test_2 = list(map(lambda s: s.strip(), test_2))
    test_2 = np.array(test_2)
    old_vectors_2.append(test_2)

In [91]:
old_array = np.array(old_vectors_2)

In [126]:
old_vectors_df_2 = old_vectorized.drop(['token','vector'], axis=1)
old_vectors_df_2.head

<bound method NDFrame.head of             0         1         2         3         4         5         6  \
0    0.009340 -0.011540 -0.005011  0.170543 -0.070042 -0.022673  0.035479   
1    0.073121 -0.047603  0.091147  0.162190 -0.052149 -0.108441 -0.028311   
2    0.013254  0.050222 -0.089445  0.083483 -0.044747 -0.065899  0.019941   
3    0.017232 -0.138262 -0.069550  0.161501 -0.024385 -0.121781 -0.035268   
4   -0.056970  0.020302 -0.024677 -0.048865  0.064803 -0.033035 -0.029016   
..        ...       ...       ...       ...       ...       ...       ...   
936 -0.021196 -0.011311 -0.073023  0.030679  0.117935  0.040947  0.024909   
937 -0.028905  0.093232 -0.058349 -0.013379 -0.086675  0.104365  0.064961   
938 -0.074280  0.093354 -0.145046  0.006174  0.000712  0.073791  0.088202   
939 -0.045976 -0.006391 -0.056196 -0.057425 -0.018857 -0.025275  0.079501   
940  0.014605  0.167768 -0.082776 -0.017683 -0.045675  0.065280  0.051255   

            7         8         9  ...       

In [ ]:
## inroads data

In [104]:
inroads_vocab = list()

for i in range(0,len(inroads_vectorized)):
    test = inroads_vectorized.iloc[i].tokens
    inroads_vocab.append(test)

inroads_vocab_df = pd.DataFrame(inroads_vocab, columns=["token"])

In [105]:
inroads_vectors_2 = list()

for i in range(0,len(inroads_vectorized)):
    test = inroads_vectorized.iloc[i].vector
    test_2 = str(test)[1:-1] 
    test_2 = Convert(test_2)
    while("" in test_2) : 
        test_2.remove("")
    test_2 = list(map(lambda s: s.strip(), test_2))
    test_2 = np.array(test_2)
    inroads_vectors_2.append(test_2)

In [106]:
import numpy as np
inroads_array = np.array(inroads_vectors_2)

In [108]:
inroads_vectors_df_2 = inroads_vectorized.drop(['tokens','vector'], axis=1)
inroads_vectors_df_2.head

<bound method NDFrame.head of               0         1         2         3         4         5         6  \
0      0.188549 -0.051055 -0.051035  0.012129  0.018650 -0.108756 -0.054335   
1     -0.089151 -0.032444  0.066067 -0.083048 -0.025339 -0.115493 -0.082126   
2      0.080989  0.071457 -0.036697  0.051603 -0.098144 -0.004233  0.047631   
3      0.048181 -0.024815 -0.042576  0.044887 -0.072843 -0.014483 -0.092501   
4     -0.002084  0.084489  0.014706  0.073587 -0.098434 -0.009841 -0.012736   
...         ...       ...       ...       ...       ...       ...       ...   
10056 -0.014218 -0.024084  0.081345 -0.069396 -0.051680  0.006385 -0.056575   
10057 -0.073358 -0.063885  0.057451 -0.059011 -0.102870  0.027484 -0.090268   
10058 -0.082558  0.043012 -0.118982 -0.168650  0.035012  0.004704  0.010404   
10059  0.010403 -0.001597  0.022658  0.142871  0.002488 -0.039659 -0.024233   
10060  0.013340  0.024281 -0.089154  0.055416  0.031318  0.012631  0.109731   

              7      

In [92]:
def clustering_on_wordvecs(word_vectors, num_clusters):
    # Initalize a k-means object and use it to extract centroids
    kmeans_clustering = KMeans(n_clusters = num_clusters, init='k-means++');
    idx = kmeans_clustering.fit_predict(word_vectors);
    
    return kmeans_clustering, kmeans_clustering.cluster_centers_, idx;

In [93]:
def get_top_words(index2word, k, centers, wordvecs):
    tree = KDTree(wordvecs);
#Closest points for each Cluster center is used to query the closest 20 points to it.
    closest_points = [tree.query(np.reshape(x, (1, -1)), k=k) for x in centers];
    closest_words_idxs = [x[1] for x in closest_points];
#Word Index is queried for each position in the above array, and added to a Dictionary.
    closest_words = {};
    for i in range(0, len(closest_words_idxs)):
        closest_words['Cluster #' + str(i)] = [index2word[j] for j in closest_words_idxs[i][0]]
#A DataFrame is generated from the dictionary.
    df = pd.DataFrame(closest_words);
    df.index = df.index+1
    return df, tree;

In [94]:
kmodel,centers,clusters = clustering_on_wordvecs(old_vectors_2, 9);
centroid_map = dict(zip(old_vocab, clusters));

In [109]:
predicted_cluster = {}

for i in range(0, len(inroads_vectors_2)):
    word = inroads_vocab[i]
    temp = kmodel.predict(inroads_vectors_2[i].reshape(1,-1))[0]
    predicted_cluster.update({word:temp})

In [110]:
predicted_cluster

{'research': 3,
 'article': 7,
 'personality': 7,
 'social': 2,
 'psychology': 7,
 'benevolent': 3,
 'sexism': 2,
 'attitudes': 2,
 'toward': 6,
 'bulletin': 5,
 'society': 7,
 'motherhood': 2,
 'reproductive': 2,
 'right': 6,
 'reprint': 4,
 'multi': 3,
 'study': 7,
 'longitudinal': 3,
 'examination': 7,
 'abortion': 0,
 'huang': 0,
 'paul': 4,
 'davies': 5,
 'chris': 5,
 'sibley': 5,
 'danny': 5,
 'osborne': 4,
 'abstract': 7,
 'although': 2,
 'ideology': 7,
 'that': 7,
 'highly': 7,
 'revere': 4,
 'women': 2,
 'conform': 2,
 'traditional': 2,
 'gender': 2,
 'cloak': 4,
 'superficially': 4,
 'positive': 2,
 'tone': 4,
 'place': 2,
 'upon': 7,
 'pedestal': 5,
 'inherently': 7,
 'restrictive': 7,
 'accordingly': 7,
 'because': 2,
 'paternalistic': 1,
 'beliefs': 7,
 'associate': 2,
 'with': 2,
 'base': 3,
 'idealization': 4,
 'roles': 2,
 'which': 7,
 'include': 3,
 'should': 7,
 'predict': 6,
 'people': 2,
 'data': 3,
 'from': 6,
 'nationwide': 3,
 'panel': 3,
 'show': 2,
 'hostile': 

In [115]:
top_words,tree = get_top_words(old_vocab, 20, centers, old_array);
print(top_words)

       Cluster #0       Cluster #1    Cluster #2     Cluster #3 Cluster #4  \
1    mifepristone         violence         women         health       love   
2     misoprostol             rape        stigma         access       feel   
3        surgical      prosecution        gender      education      grief   
4       pregnancy            abuse    motherhood           fund    sadness   
5   complications           murder      children           care     sorrow   
6         uterine             harm        sexual     healthcare       pity   
7      dilatation        injustice        mother        support      happy   
8          uterus          cruelty  reproduction      providers    comfort   
9      ultrasound           police    individual       outreach      sleep   
10       cervical           attack        social         sector       fear   
11        surgery            crime          male  organizations   frighten   
12    antibiotics         criminal  reproductive         policy 

In [ ]:
## sagemaker deploy

In [119]:
from sagemaker import get_execution_role
role = get_execution_role()

In [130]:
s3_client = boto3.client('s3')
bucket='inroads-test-bucket1'

In [134]:
old_vectors_df_2.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '290', '291', '292', '293', '294', '295', '296', '297', '298', '299'],
      dtype='object', length=300)

In [141]:
train_data = old_array.astype('float32')

In [142]:
from sagemaker import KMeans

num_clusters = 9
kmeans = KMeans(role=role,
                train_instance_count=1,
                train_instance_type='ml.c4.xlarge',
                output_path='s3://'+ bucket +'/kmeans/',              
                k=num_clusters)

In [143]:
%%time
kmeans.fit(kmeans.record_set(train_data))

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2020-08-20 18:55:17 Starting - Starting the training job...
2020-08-20 18:55:19 Starting - Launching requested ML instances......
2020-08-20 18:56:37 Starting - Preparing the instances for training.........
2020-08-20 18:58:08 Downloading - Downloading input data...
2020-08-20 18:58:50 Training - Training image download completed. Training in progress.
2020-08-20 18:58:50 Uploading - Uploading generated training model.Docker entrypoint called with argument(s): train
Running default environment configuration script
[08/20/2020 18:58:47 INFO 139919176947520] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'_enable_profiler': u'false', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'local_lloyd_num_trials': u'auto', u'_log_level': u'info', u'_kvstore': u'auto', u'local_lloyd_init_method': u'kmeans++', u'force_dense': u'true', u'epochs': u'1', u'init_method': u'random', u'local_lloyd_tol': u'0.0001', u'local_ll

In [ ]:
%%time
kmeans_predictor = kmeans.deploy(initial_instance_count=1, 
                                 instance_type='ml.t2.medium')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-----------------------

KeyboardInterrupt: 

In [ ]:
%%time
result=kmeans_predictor.predict(train_data)

NameError: name 'kmeans_predictor' is not defined

In [ ]:
cluster_labels = [r.label['closest_cluster'].float32_tensor.values[0] for r in result]

In [ ]:
pd.DataFrame(cluster_labels)[0].value_counts()

In [ ]:
ax=plt.subplots(figsize=(6,3))
ax=sns.distplot(cluster_labels, kde=False)
title="Histogram of Cluster Counts"
ax.set_title(title, fontsize=12)
plt.show()

In [ ]:
job_name = kmeans.latest_training_job.name
model_key = "kmeans/" + job_name + "/output/model.tar.gz"

boto3.resource('s3').Bucket(bucket).download_file(model_key, 'model.tar.gz')
os.system('tar -zxvf model.tar.gz')

In [147]:
import sagemaker
sagemaker.Session().delete_endpoint(kmeans_predictor.endpoint)

NameError: name 'kmeans_predictor' is not defined